In [2]:
import numpy as np

In [21]:
%%capture
!pip install datasets transformers
!pip install accelerate -U
!pip install opencv-python
!pip install scikit-learn

In [7]:
project_name = 'FunnyUpdate'
name = 'vit_huge'

In [8]:
model_path = 'google/vit-huge-patch14-224-in21k'

In [13]:
# Critical imports
import os
import numpy as np
import pandas as pd
from PIL import Image
import random
import cv2
import copy
import torch
import torchvision
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import shutil

In [14]:
from transformers import AutoImageProcessor
model_name_or_path = model_path
image_processor  = AutoImageProcessor.from_pretrained(model_name_or_path)
image_processor

/home/vedaant/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [15]:
%%capture
!pip install huggingface-hub

In [2]:
import huggingface_hub
huggingface_hub.login(hugging_face_login_key))

SyntaxError: unmatched ')' (2081314194.py, line 2)

In [17]:
from datasets import load_dataset

train_dataset = load_dataset("imagefolder", data_dir="NewDataset5/Train/")
valid_dataset = load_dataset("imagefolder", data_dir="NewDataset5/Valid/")
test_dataset = load_dataset("imagefolder", data_dir="NewDataset5/Test/")
train_dataset, test_dataset, valid_dataset

Resolving data files: 100%|█████████████████████████████████████████████████████| 1516/1516 [00:00<00:00, 253147.74it/s]
Extracting data files: 0it [00:00, ?it/s]
Generating train split: 1516 examples [00:00, 14150.33 examples/s]
Resolving data files: 100%|███████████████████████████████████████████████████████| 483/483 [00:00<00:00, 242587.57it/s]
Extracting data files: 0it [00:00, ?it/s]
Generating train split: 483 examples [00:00, 17266.84 examples/s]
Resolving data files: 100%|███████████████████████████████████████████████████████| 471/471 [00:00<00:00, 180215.03it/s]
Extracting data files: 0it [00:00, ?it/s]
Generating train split: 471 examples [00:00, 11254.71 examples/s]


(DatasetDict({
     train: Dataset({
         features: ['image', 'label'],
         num_rows: 1516
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['image', 'label'],
         num_rows: 471
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['image', 'label'],
         num_rows: 483
     })
 }))

In [18]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = image_processor([x for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['labels'] = example_batch['label']
    return inputs

prepared_train_ds = train_dataset.with_transform(transform)
prepared_valid_ds = valid_dataset.with_transform(transform)
prepared_test_ds = test_dataset.with_transform(transform)
prepared_train_ds, prepared_valid_ds, prepared_test_ds

(DatasetDict({
     train: Dataset({
         features: ['image', 'label'],
         num_rows: 1516
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['image', 'label'],
         num_rows: 483
     })
 }),
 DatasetDict({
     train: Dataset({
         features: ['image', 'label'],
         num_rows: 471
     })
 }))

In [22]:
import numpy as np
from datasets import load_metric

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

In [24]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

In [25]:
from accelerate import Accelerator
accelerator = Accelerator()

In [33]:
BATCH_SIZE = 4
train_dataloader = DataLoader(prepared_train_ds['train'], batch_size = BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
valid_dataloader = DataLoader(prepared_valid_ds['train'], batch_size = BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(prepared_test_ds['train'], batch_size = BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

In [35]:
from transformers import AutoImageProcessor, ViTForImageClassification
labels = prepared_train_ds['train'].features['label'].names
model = ViTForImageClassification.from_pretrained(model_name_or_path, num_labels=len(labels), id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}, ignore_mismatched_sizes=True)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.1, weight_decay=0.001)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-huge-patch14-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
model.device

device(type='cuda', index=0)

In [ ]:
BATCH_SIZE = #BATCH_SIZE
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./"+name+project_name,
  per_device_train_batch_size=BATCH_SIZE,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=1e-5,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=True,
  report_to='tensorboard',
  load_best_model_at_end=True,
  optim='adamw_torch',
  weight_decay=0.01
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_train_ds["train"],
    eval_dataset=prepared_valid_ds["train"],
    tokenizer=image_processor,
)

In [ ]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

In [ ]:
metrics = trainer.predict(prepared_test_ds['train'])
trainer.log_metrics("test", metrics.metrics)
trainer.save_metrics("test", metrics.metrics)

In [ ]:
predictions_tensor = torch.tensor(metrics.predictions)
preds = torch.sigmoid(predictions_tensor)
print(preds.shape)
preds = torch.argmax(preds, dim=1)
preds, preds.shape

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [ ]:
print(accuracy_score(preds, metrics.label_ids), f1_score(preds, metrics.label_ids))
pd.DataFrame(confusion_matrix(preds, metrics.label_ids), columns=['Test Funny', 'Test Not Funny'], index=['Pred Funny','Pred Not Funny'])

In [ ]:
metrics = trainer.predict(prepared_train_ds['train'])
predictions_tensor = torch.tensor(metrics.predictions)
preds = torch.sigmoid(predictions_tensor)
print(preds.shape)
preds = torch.argmax(preds, dim=1)
preds, preds.shape
print(accuracy_score(preds, metrics.label_ids), f1_score(preds, metrics.label_ids))
pd.DataFrame(confusion_matrix(preds, metrics.label_ids), columns=['Train Funny', 'Train Not Funny'], index=['Pred Funny','Pred Not Funny'])

In [ ]:
metrics = trainer.predict(prepared_valid_ds['train'])
predictions_tensor = torch.tensor(metrics.predictions)
preds = torch.sigmoid(predictions_tensor)
print(preds.shape)
preds = torch.argmax(preds, dim=1)
preds, preds.shape
print(accuracy_score(preds, metrics.label_ids), f1_score(preds, metrics.label_ids))
pd.DataFrame(confusion_matrix(preds, metrics.label_ids), columns=['Valid Funny', 'Valid Not Funny'], index=['Pred Funny','Pred Not Funny'])